<p style="text-align: center; font-size: 25px;"> IBI5031 - Aprendizagem de Máquina para Bioinformática<br>
Terceira Lista de Exercı́cios<br>
Docente: Marcelo da Silva Reis<br>
Laboratório de Ciclo Celular, Instituto Butantan<br>
Programa de Pós-Graduação Interunidades em Bioinformática da USP </p>

<hr>

<p style="text-align: center; font-size: 20px;"> Nome: Guilherme Paschoalini <br>
    N USP: 11515572 </p>

<hr>

Questão 1. (10 pontos) Entre no repositório UCI Machine Learning Repository e baixe o conjunto de dados ”Parkinsons”:
    
archive.ics.uci.edu/ml/datasets/parkinsons

Este conjunto de dados foi gerado para classificar pessoas como portadoras ou não da doença de Parkinson a partir de padrões vocais; portanto, é um conjunto de dados de
classificação binária, com uma amostra de 197 pontos e 23 caracterı́sticas. Utilizando as máquinas de suporte vetorial (SVMs) do scikit learn, cuja documentação está disponı́vel em:

scikit-learn.org/stable/modules/svm.html

classifique Parkinsons utilizando os modelos SVMs disponı́veis nesse módulo. Considere o uso de diferentes kernels (transformações não-lineares feitas de forma implı́cita) disponı́veis no scikit learn:
    
- linear;
- polinomial;
- função de base radial (RBF);
- sigmóide (implementado com a tangente hiperbólica).

Faça seleção de modelos utilizando validação cruzada e diferentes valores de parâmetros para os diferentes kernels (e.g., o parâmetro d que determina o grau do kernel polinomial).
Organize seus resultados finais em uma matriz de confusão.

<hr>

#### Bibliotecas necessárias para execução da tarefa 

In [2]:
import pandas as pd
import pylab as pl
import numpy as np
import scipy.optimize as opt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
import sklearn
%matplotlib inline 
import matplotlib.pyplot as plt
from sklearn.svm import SVC
import pandas as pd
import numpy as np
from sklearn import model_selection
from sklearn.metrics import classification_report, confusion_matrix
import itertools

### Importação dos dados
#### Os dados serão importados diretamente do link fornecido pela Uci, sendo assim não há necessidade de download local

In [3]:
data = pd.read_table('https://archive.ics.uci.edu/ml/machine-learning-databases/parkinsons/parkinsons.data', sep=',')

#### Após isso podemos verificar as informações contidas por meio de um agrupado de linhas e colunas, apesar do próprio dataframe já nos exibir a quantidade de linhas e colunas podemos utilizar a função shape para confirmar
#### Nossa base de dados possui:
- 195 linhas (pacientes e ou participantes)
- 24 colunas (variáveis coletadas)

In [4]:
data

,name,MDVP:Fo(Hz),MDVP:Fhi(Hz),MDVP:Flo(Hz),MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP,MDVP:Shimmer,...,Shimmer:DDA,NHR,HNR,status,RPDE,DFA,spread1,spread2,D2,PPE
0,phon_R01_S01_1,119.992,157.302,74.997,0.00784,0.00007,0.00370,0.00554,0.01109,0.04374,...,0.06545,0.02211,21.033,1,0.414783,0.815285,-4.813031,0.266482,2.301442,0.284654
1,phon_R01_S01_2,122.400,148.650,113.819,0.00968,0.00008,0.00465,0.00696,0.01394,0.06134,...,0.09403,0.01929,19.085,1,0.458359,0.819521,-4.075192,0.335590,2.486855,0.368674
2,phon_R01_S01_3,116.682,131.111,111.555,0.01050,0.00009,0.00544,0.00781,0.01633,0.05233,...,0.08270,0.01309,20.651,1,0.429895,0.825288,-4.443179,0.311173,2.342259,0.332634
3,phon_R01_S01_4,116.676,137.871,111.366,0.00997,0.00009,0.00502,0.00698,0.01505,0.05492,...,0.08771,0.01353,20.644,1,0.434969,0.819235,-4.117501,0.334147,2.405554,0.368975
4,phon_R01_S01_5,116.014,141.781,110.655,0.01284,0.00011,0.00655,0.00908,0.01966,0.06425,...,0.10470,0.01767,19.649,1,0.417356,0.823484,-3.747787,0.234513,2.332180,0.410335
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,phon_R01_S50_2,174.188,230.978,94.261,0.00459,0.00003,0.00263,0.00259,0.00790,0.04087,...,0.07008,0.02764,19.517,0,0.448439,0.657899,-6.538586,0.121952,2.657476,0.133050
191,phon_R01_S50_3,209.516,253.017,89.488,0.00564,0.00003,0.00331,0.00292,0.00994,0.02751,...,0.04812,0.01810,19.147,0,0.431674,0.683244,-6.195325,0.129303,2.784312,0.168895
192,phon_R01_S50_4,174.688,240.005,74.287,0.01360,0.00008,0.00624,0.00564,0.01873,0.02308,...,0.03804,0.10715,17.883,0,0.407567,0.655683,-6.787197,0.158453,2.679772,0.131728
193,phon_R01_S50_5,198.764,396.961,74.904,0.00740,0.00004,0.00370,0.00390,0.01109,0.02296,...,0.03794,0.07223,19.020,0,0.451221,0.643956,-6.744577,0.207454,2.138608,0.123306


In [7]:
data.shape

(195, 24)

#### Com o uso da função describe podemos resumidamente verificar algumas caracteristicas sobre as variáveis, como: média, desvio padrão, quartis e o valor minímo e máximo

In [5]:
data.describe()

,MDVP:Fo(Hz),MDVP:Fhi(Hz),MDVP:Flo(Hz),MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP,MDVP:Shimmer,MDVP:Shimmer(dB),...,Shimmer:DDA,NHR,HNR,status,RPDE,DFA,spread1,spread2,D2,PPE
count,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000,...,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000,195.000000
mean,154.228641,197.104918,116.324631,0.006220,0.000044,0.003306,0.003446,0.009920,0.029709,0.282251,...,0.046993,0.024847,21.885974,0.753846,0.498536,0.718099,-5.684397,0.226510,2.381826,0.206552
std,41.390065,91.491548,43.521413,0.004848,0.000035,0.002968,0.002759,0.008903,0.018857,0.194877,...,0.030459,0.040418,4.425764,0.431878,0.103942,0.055336,1.090208,0.083406,0.382799,0.090119
min,88.333000,102.145000,65.476000,0.001680,0.000007,0.000680,0.000920,0.002040,0.009540,0.085000,...,0.013640,0.000650,8.441000,0.000000,0.256570,0.574282,-7.964984,0.006274,1.423287,0.044539
25%,117.572000,134.862500,84.291000,0.003460,0.000020,0.001660,0.001860,0.004985,0.016505,0.148500,...,0.024735,0.005925,19.198000,1.000000,0.421306,0.674758,-6.450096,0.174351,2.099125,0.137451
50%,148.790000,175.829000,104.315000,0.004940,0.000030,0.002500,0.002690,0.007490,0.022970,0.221000,...,0.038360,0.011660,22.085000,1.000000,0.495954,0.722254,-5.720868,0.218885,2.361532,0.194052
75%,182.769000,224.205500,140.018500,0.007365,0.000060,0.003835,0.003955,0.011505,0.037885,0.350000,...,0.060795,0.025640,25.075500,1.000000,0.587562,0.761881,-5.046192,0.279234,2.636456,0.252980
max,260.105000,592.030000,239.170000,0.033160,0.000260,0.021440,0.019580,0.064330,0.119080,1.302000,...,0.169420,0.314820,33.047000,1.000000,0.685151,0.825288,-2.434031,0.450493,3.671155,0.527367


#### A função dtypes permite verificar o tipo de cada variável, essa informação é essencial, diferentes tipos de váriaveis podem requerer formas distintas na manipulação dos dados 

In [13]:
data.dtypes

name                 object
MDVP:Fo(Hz)         float64
MDVP:Fhi(Hz)        float64
MDVP:Flo(Hz)        float64
MDVP:Jitter(%)      float64
MDVP:Jitter(Abs)    float64
MDVP:RAP            float64
MDVP:PPQ            float64
Jitter:DDP          float64
MDVP:Shimmer        float64
MDVP:Shimmer(dB)    float64
Shimmer:APQ3        float64
Shimmer:APQ5        float64
MDVP:APQ            float64
Shimmer:DDA         float64
NHR                 float64
HNR                 float64
status                int64
RPDE                float64
DFA                 float64
spread1             float64
spread2             float64
D2                  float64
PPE                 float64
dtype: object

#### Finalmente podemos verificar em especial a variável desfecho, binária, que indica se o participante tem (classe 1) ou não (classe 0) a condição
#### Temos:
- 147 classe 1 (75,4%)
- 048 classe 0 (24,6%)

In [16]:
data.status.value_counts()

1    147
0     48
Name: status, dtype: int64

<hr>

## Construção dos modelos preditivos

#### Como estamos trabalhando com um modelo de aprendizado supervisionado primeiramente precisamos dividir os dados em dois subconjuntos, teste e treino
#### Primeiro vamos criar um dataframe chamado 'feature_df' removendo deste as variáveis nome (variável sem necessidade no modelo) e status (desfecho desejado), após isso vamos criar um segundo dataframe somente com a informação do desfecho, em ambos os casos iremos converter o dataframe para uma variável do tipo array com o uso da biblioteca Numpy.

In [82]:
feature_df = data.drop(['name', 'status'], axis=1)

# independent variable
X = np.asarray(feature_df)

# dependent variable
data['status'] = data['status'].astype('int')
y = np.asarray(data['status'])

#### Após a etapa anterior, com as novas variáveis definidas iremos criar as bases de teste e treino, a base de treino terá 80% dos dados, logo a base de teste 20% dos dados, para essa tarefa utilizaremos a função train_test_split parte do pacote Sklearn, uma semente aleatória será definida com o valor de 101 permitindo assim a reprodutibilidade dos resultados encontrados aqui.

In [83]:
X_train, X_test, y_train, y_test = model_selection.train_test_split( X, y, test_size=0.2, random_state=101)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

Train set: (156, 22) (156,)
Test set: (39, 22) (39,)


#### Para o primeiro modelo preditivo utilizaremos o kernel RBF (função de base radial), inclusive caso essa opção não seja definida no código esse kernel será assumido como default

#### Para a seleção dos modelos com uso de validação cruzada e diferentes valores de ajustes nos parâmetros, irei utilizar a função GridSearchCV também presente no pacote Sklearn
#### Essa função permite definir um grid de parâmetros a serem testados, todas as combinações entre eles serão executadas, o melhor modelo é definido a partir da métrica de avaliação definida, nesse caso será utilizado balanced_accuracy. A acurácia balanceada de acordo com a documentação do Sklearn é definida como a média de evocação obtida em cada classe, em outras palavras é dada pela soma da sensibilidade + especificidade / 2. Esse parâmetro será igualmente aplicada para os 4 kernels testados.
#### A validação cruzada será utilizada com k definido em 5, novamente a semente aleatória foi definida com o valor de 101 para reprodutibilidade e o parâmetro n_jobs permite definir quantos núcleos de processamento serão utilizados na tafera.
#### Os parâmetros definidos no grid, além do kernel RBF foram:
- C (regularização), com valores de 0.1 a 100
- gamma (coeficiente do kernel - não se aplica quando kernel é poliminial), com valores de 0.0001-1 e escalável

#### Os quatros modelos seguem a mesma lógica e sequência no desenvolvimento e teste

<hr>

### RBF (função de base radial)

In [84]:
param_gridSVM = {'C': [0.1, 1, 10, 100], 
                 'gamma': ['scale', 1, 0.1, 0.001, 0.0001],
                 'kernel': ['rbf']}  

gridSVM = GridSearchCV(cv=5, estimator=SVC(class_weight='balanced', random_state=101), n_jobs=8, 
                       param_grid=param_gridSVM, scoring='balanced_accuracy') 

gridSVM.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=SVC(class_weight='balanced', random_state=101),
             n_jobs=8,
             param_grid={'C': [0.1, 1, 10, 100],
                         'gamma': ['scale', 1, 0.1, 0.001, 0.0001],
                         'kernel': ['rbf']},
             scoring='balanced_accuracy')

### Melhores parâmetros e estimador encontrados para kernel RBF
- C = 100
- gamma = 0.0001

In [85]:
print(gridSVM.best_params_)

{'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'}


In [86]:
print(gridSVM.best_estimator_)

SVC(C=100, class_weight='balanced', gamma=0.0001, random_state=101)


#### Predição com as informações da base de testes

In [87]:
y_pred_svm = gridSVM.predict(X_test)

### Criação da matriz de confusão com os resultados
#### Irei utilizar a função confusion_matrix também presente no Sklearn

In [88]:
labels = np.unique(y_test)

cm = pd.DataFrame(confusion_matrix(y_test, y_pred_svm, labels=labels), index=labels, columns=labels)
cm

,0,1
0,11,0
1,3,25


### Neste modelo com uso de kernel RBF temos as seguintes métricas

- Sensibilidade: 1
- Especificidade: 0.78
- Acurácia: 92.3%
- Valor Preditivo Positivo: 0.89
- Valor Preditivo Negativo: 1

<hr>

### Linear

In [62]:
param_gridSVM = {'C': [0.1, 1, 10, 100], 
                 'gamma': ['scale', 1, 0.1, 0.001, 0.0001],
                 'kernel': ['linear']}

gridSVM = GridSearchCV(cv=5, estimator=SVC(class_weight='balanced', random_state=101), n_jobs=8, 
                       param_grid=param_gridSVM, scoring='balanced_accuracy') 

gridSVM.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=SVC(class_weight='balanced', random_state=101),
             n_jobs=8,
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': ['scale', 'auto', 1, 0.1, 0.01, 0.001,
                                   0.0001],
                         'kernel': ['linear']},
             scoring='balanced_accuracy')

In [63]:
print(gridSVM.best_params_)

{'C': 1, 'gamma': 'scale', 'kernel': 'linear'}


In [64]:
print(gridSVM.best_estimator_) 

SVC(C=1, class_weight='balanced', kernel='linear', random_state=101)


In [65]:
y_pred_svm = gridSVM.predict(X_test)

In [66]:
labels = np.unique(y_test)

cm = pd.DataFrame(confusion_matrix(y_test, y_pred_svm, labels=labels), index=labels, columns=labels)
cm

,0,1
0,9,2
1,3,25


### Neste modelo com uso de kernel LINEAR temos as seguintes métricas

- Sensibilidade: 0.92
- Especificidade: 0.75
- Acurácia: 87.1%
- Valor Preditivo Positivo: 0.89
- Valor Preditivo Negativo: 0.81

<hr>

### Polimonial
#### Diferente dos anteriores o kernel polimonial não utiliza o parâmetro gamma e sim o degree (grau), para buscar o melhor ajuste do modelo utilizei os valores de 2, 3 e 4.

In [105]:
param_gridSVM = {'C': [ 1, 10, 100, 1000], 
                 'degree':[2,3,4],
                 'kernel': ['poly']}

gridSVM = GridSearchCV(cv=5, estimator=SVC(class_weight='balanced', random_state=101), n_jobs=8, 
                       param_grid=param_gridSVM, scoring='balanced_accuracy') 

gridSVM.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=SVC(class_weight='balanced', random_state=101),
             n_jobs=8,
             param_grid={'C': [1, 10, 100, 1000], 'degree': [2, 3, 4],
                         'kernel': ['poly']},
             scoring='balanced_accuracy')

In [106]:
print(gridSVM.best_params_)

{'C': 1000, 'degree': 4, 'kernel': 'poly'}


In [107]:
print(gridSVM.best_estimator_)

SVC(C=1000, class_weight='balanced', degree=4, kernel='poly', random_state=101)


In [108]:
y_pred_svm = gridSVM.predict(X_test)

In [109]:
labels = np.unique(y_test)

cm = pd.DataFrame(confusion_matrix(y_test, y_pred_svm, labels=labels), index=labels, columns=labels)
cm

,0,1
0,8,3
1,1,27


### Neste modelo com uso de kernel POLIMONIAL temos as seguintes métricas

- Sensibilidade: 0.9
- Especificidade: 0.88
- Acurácia: 87.9%
- Valor Preditivo Positivo: 0.96
- Valor Preditivo Negativo: 0.72

<hr>

### Sigmoid

In [69]:
param_gridSVM = {'C': [0.1, 1, 10, 100], 
                 'gamma': ['scale', 1, 0.1, 0.001, 0.0001],
                 'kernel': ['sigmoid']}

gridSVM = GridSearchCV(cv=5, estimator=SVC(class_weight='balanced', random_state=101), n_jobs=8, 
                       param_grid=param_gridSVM, scoring='balanced_accuracy') 

gridSVM.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=SVC(class_weight='balanced', random_state=101),
             n_jobs=8,
             param_grid={'C': [0.1, 1, 10, 100],
                         'gamma': ['scale', 1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['sigmoid']},
             scoring='balanced_accuracy')

In [70]:
print(gridSVM.best_params_)

{'C': 1, 'gamma': 0.0001, 'kernel': 'sigmoid'}


In [71]:
print(gridSVM.best_estimator_)

SVC(C=1, class_weight='balanced', gamma=0.0001, kernel='sigmoid',
    random_state=101)


In [72]:
y_pred_svm = gridSVM.predict(X_test)

In [73]:
labels = np.unique(y_test)

cm = pd.DataFrame(confusion_matrix(y_test, y_pred_svm, labels=labels), index=labels, columns=labels)

cm

,0,1
0,0,11
1,0,28


### Neste modelo com uso de kernel SIGMOID temos as seguintes métricas

- Sensibilidade: 1
- Especificidade: 0
- Acurácia: 71.7%
- Valor Preditivo Positivo: 0.71
- Valor Preditivo Negativo: 0

<hr>

## Considerações Finais

### Levando em consideração a acurácia de cada teste, o modelo com kernel RBF (função de base radial) que obteve 92.3% poderia ser definido como o melhor modelo. Entretanto essa questão deve levar em consideração o tipo de teste que se deseja (triagem ou diagnóstico), somente assim podemos definir parâmetros corretos que atendam as necessidade locais para o problema alvo.